In [5]:
!pip install cirq

In [6]:
import cirq
import numpy as np
import pandas as pd
import yfinance as yf
import sympy
from collections import Counter

# --- 1. Data Ingestion ---
def get_stock_data(tickers, start_date, end_date):
    """
    Fetches historical adjusted close prices for given tickers.
    Calculates daily returns, expected returns, and covariance matrix.
    """
    print(f"Fetching data for tickers: {tickers} from {start_date} to {end_date}...")
    # Explicitly set auto_adjust=False to ensure 'Adj Close' column is available
    data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False)['Adj Close']


    # Calculate daily returns
    returns = data.pct_change().dropna()

    # Calculate expected returns (mean daily return)
    expected_returns = returns.mean()

    # Calculate covariance matrix
    covariance_matrix = returns.cov()

    print("Data fetched and processed successfully.")
    return expected_returns, covariance_matrix, returns.index

# --- 2. Problem Formulation (QUBO to Ising) ---
def create_ising_hamiltonian(expected_returns, covariance_matrix, num_assets_to_select, risk_aversion_param=0.5, penalty_param=10.0):
    """
    Formulates the portfolio optimization problem as an Ising Hamiltonian.

    Args:
        expected_returns (pd.Series): Expected returns for each asset.
        covariance_matrix (pd.DataFrame): Covariance matrix of asset returns.
        num_assets_to_select (int): The number of assets to select in the portfolio (k).
        risk_aversion_param (float): Lambda parameter for risk aversion.
        penalty_param (float): Gamma_P parameter for the constraint penalty.

    Returns:
        tuple: (h_coeffs, J_coeffs) for the Ising Hamiltonian.
               h_coeffs (dict): Linear coefficients for Z_i terms.
               J_coeffs (dict): Quadratic coefficients for Z_i Z_j terms.
    """
    n = len(expected_returns)

    # Initialize QUBO coefficients
    L = {}  # Linear coefficients for x_i
    Q = {}  # Quadratic coefficients for x_i x_j

    # Calculate L_i and Q_ij from the QUBO formulation
    # H_QUBO = sum_i L_i x_i + sum_{i<j} Q_{ij} x_i x_j + constant

    for i in range(n):
        # L_i = -mu_i + gamma_P(1-2k) + lambda * Sigma_ii
        L[i] = -expected_returns.iloc[i] + penalty_param * (1 - 2 * num_assets_to_select) + risk_aversion_param * covariance_matrix.iloc[i, i]

        for j in range(i + 1, n):
            # Q_ij = 2 * lambda * Sigma_ij + 2 * gamma_P
            Q[(i, j)] = 2 * risk_aversion_param * covariance_matrix.iloc[i, j] + 2 * penalty_param

    # Convert QUBO coefficients to Ising coefficients (h_i, J_ij)
    # H_Ising = sum_i h_i Z_i + sum_{i<j} J_ij Z_i Z_j

    h_coeffs = {}
    J_coeffs = {}

    for i in range(n):
        h_i_val = L[i] / 2.0
        # Add contributions from Q_ij terms where x_i is involved
        for j in range(n):
            if i < j:
                h_i_val += Q[(i, j)] / 4.0
            elif j < i:
                h_i_val += Q[(j, i)] / 4.0 # Q_ji is the same as Q_ij
        h_coeffs[i] = h_i_val

    for (i, j), q_ij_val in Q.items():
        J_coeffs[(i, j)] = q_ij_val / 4.0

    print("Ising Hamiltonian coefficients generated.")
    return h_coeffs, J_coeffs

# --- 3. QAOA Circuit Construction ---
def problem_layer(qubits, gamma, h_coeffs, J_coeffs):
    """
    Applies the problem Hamiltonian layer (exp(-i * gamma * H_P)).
    H_P = sum_i h_i Z_i + sum_{i<j} J_ij Z_i Z_j
    """
    # Z terms
    for i, h_val in h_coeffs.items():
        if abs(h_val) > 1e-9: # Only apply if coefficient is significant
            yield cirq.Z(qubits[i])**(-2 * gamma * h_val / np.pi) # Z**exponent corresponds to exp(-i * pi * exponent / 2 * Z)
                                                                 # We want exp(-i * gamma * h_val * Z), so exponent = 2 * gamma * h_val / pi

    # ZZ terms
    for (i, j), J_val in J_coeffs.items():
        if abs(J_val) > 1e-9: # Only apply if coefficient is significant
            yield cirq.CZ(qubits[i], qubits[j])**(-2 * gamma * J_val / np.pi) # CZ**exponent corresponds to exp(-i * pi * exponent / 2 * Z_i Z_j)
                                                                            # We want exp(-i * gamma * J_val * Z_i Z_j), so exponent = 2 * gamma * J_val / pi

def mixer_layer(qubits, beta):
    """
    Applies the mixer Hamiltonian layer (exp(-i * beta * H_M)).
    H_M = sum_i X_i
    """
    for q in qubits:
        yield cirq.X(q)**(-2 * beta / np.pi) # X**exponent corresponds to exp(-i * pi * exponent / 2 * X)
                                             # We want exp(-i * beta * X), so exponent = 2 * beta / pi

def qaoa_circuit(qubits, h_coeffs, J_coeffs, num_layers, gammas, betas):
    """
    Constructs the full QAOA circuit.
    """
    circuit = cirq.Circuit()

    # Initial superposition state (Hadamard on all qubits)
    circuit.append(cirq.H(q) for q in qubits)

    # QAOA layers
    for layer in range(num_layers):
        circuit.append(problem_layer(qubits, gammas[layer], h_coeffs, J_coeffs))
        circuit.append(mixer_layer(qubits, betas[layer]))

    # Measure all qubits
    circuit.append(cirq.measure(*qubits, key='x'))

    print(f"QAOA circuit constructed with {num_layers} layers.")
    return circuit

# --- 4. Objective Function for QAOA ---
def calculate_ising_energy(measurements, h_coeffs, J_coeffs, qubits):
    """
    Calculates the energy of the Ising Hamiltonian for a given measurement outcome.
    Measurement is a bitstring (e.g., [0, 1, 0, 1]).
    Converts bitstring to spin values (+1 for 0, -1 for 1, or vice versa depending on convention).
    Here, we use 0 -> +1, 1 -> -1 (Z_i = (-1)^x_i)
    """
    # Convert bitstring to spin values: 0 -> +1, 1 -> -1
    # This maps computational basis |0> to spin +1 (up) and |1> to spin -1 (down)
    spin_values = np.array([1 if bit == 0 else -1 for bit in measurements])

    energy = 0.0

    # Linear terms (h_i Z_i)
    for i, h_val in h_coeffs.items():
        energy += h_val * spin_values[qubits.index(cirq.LineQubit(i))]

    # Quadratic terms (J_ij Z_i Z_j)
    for (i, j), J_val in J_coeffs.items():
        idx_i = qubits.index(cirq.LineQubit(i))
        idx_j = qubits.index(cirq.LineQubit(j))
        energy += J_val * spin_values[idx_i] * spin_values[idx_j]

    return energy

def obj_func_qaoa(result, h_coeffs, J_coeffs, qubits):
    """
    Calculates the average energy from simulation results.
    """
    # The fold_func receives the raw measurement tuple (e.g., (0, 1, 0))
    # We need to pass the current h_coeffs, J_coeffs, and qubits to it.

    # Create a lambda function to pass additional arguments to calculate_ising_energy
    energy_calculator = lambda m: calculate_ising_energy(m, h_coeffs, J_coeffs, qubits)

    energy_hist = result.histogram(key='x', fold_func=energy_calculator)

    total_energy = sum(k * v for k, v in energy_hist.items())
    total_repetitions = result.repetitions

    return total_energy / total_repetitions

# --- Main Execution ---
def run_portfolio_optimization_qaoa(
    tickers,
    start_date,
    end_date,
    num_assets_to_select,
    num_qaoa_layers,
    sweep_size=10,
    repetitions=1000
):
    """
    Runs the full QAOA-based portfolio optimization.
    """
    # 1. Data Ingestion
    expected_returns, covariance_matrix, _ = get_stock_data(tickers, start_date, end_date)

    num_assets = len(tickers)
    if num_assets < num_assets_to_select:
        print("Error: Number of assets to select cannot be more than available assets.")
        return

    # 2. Problem Formulation (Ising Hamiltonian)
    h_coeffs, J_coeffs = create_ising_hamiltonian(
        expected_returns, covariance_matrix, num_assets_to_select
    )

    # Define qubits
    qubits = cirq.LineQubit.range(num_assets)

    # Define symbolic parameters for QAOA layers
    gammas = [sympy.Symbol(f'gamma_{i}') for i in range(num_qaoa_layers)]
    betas = [sympy.Symbol(f'beta_{i}') for i in range(num_qaoa_layers)]

    # 3. QAOA Circuit Construction
    qaoa_circ = qaoa_circuit(qubits, h_coeffs, J_coeffs, num_qaoa_layers, gammas, betas)

    # 4. Optimization (Grid Search)
    simulator = cirq.Simulator()

    # Create a sweep for all parameters
    param_sweep = cirq.study.Product(
        *[cirq.Linspace(key=g.name, start=0.0, stop=np.pi, length=sweep_size) for g in gammas],
        *[cirq.Linspace(key=b.name, start=0.0, stop=np.pi/2, length=sweep_size) for b in betas]
    )

    print(f"Starting QAOA parameter sweep with {len(param_sweep)} combinations and {repetitions} repetitions per combination...")
    results = simulator.run_sweep(qaoa_circ, params=param_sweep, repetitions=repetitions)

    min_energy = float('inf')
    best_params = None
    best_result = None # Store the best result object

    for result in results:
        current_energy = obj_func_qaoa(result, h_coeffs, J_coeffs, qubits)

        if current_energy < min_energy:
            min_energy = current_energy
            best_params = result.params.param_dict
            best_result = result # Store the entire result object

    print("\nOptimization complete.")
    print(f"Minimum Energy Found: {min_energy}")
    print(f"Best Parameters: {best_params}")

    # --- 5. Result Interpretation ---
    # Analyze the measurement results for the best parameters
    # We need to find the most frequent measurement outcome (bitstring)

    # Access the measurements from the best result object
    best_measurement_counts = best_result.measurements['x']

    # Flatten the list of lists of measurements and count occurrences
    # Each inner list in best_measurement_counts is a repetition, containing an array of qubit states
    all_measurements_flat = [tuple(m) for m in best_measurement_counts]


    # Convert measurement tuples to bitstrings (e.g., (0, 1, 0) -> "010")
    bitstring_counts = Counter(["".join(map(str, m)) for m in all_measurements_flat])

    # Get the most common bitstring
    most_common_bitstring, count = bitstring_counts.most_common(1)[0]

    print(f"Most common measurement outcome (bitstring): {most_common_bitstring} (occurred {count} times)")

    # Convert bitstring to selected assets
    selected_assets_indices = [i for i, bit in enumerate(most_common_bitstring) if bit == '0'] # Assuming 0 means selected (spin +1)

    print(f"Selected asset indices (based on most common bitstring): {selected_assets_indices}")
    print("Corresponding tickers:")
    for idx in selected_assets_indices:
        print(f"- {tickers[idx]}")

    # Verify the constraint: check if the number of selected assets matches num_assets_to_select
    if len(selected_assets_indices) != num_assets_to_select:
        print(f"\nWARNING: The most common outcome selected {len(selected_assets_indices)} assets, but {num_assets_to_select} were requested.")
        print("This might indicate that the penalty_param needs to be adjusted or more QAOA layers/optimization iterations are needed.")

    return min_energy, best_params, most_common_bitstring, selected_assets_indices

# --- Example Usage ---
if __name__ == '__main__':
    # Define your stock tickers (3-8 as requested)
    my_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'] # 5 stocks
    # my_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'JPM', 'V'] # 8 stocks

    # Define the date range for historical data
    start_date = '2020-01-01'
    end_date = '2023-01-01'

    # Define the number of assets to select in the portfolio
    k_assets_to_select = 3

    # Define the number of QAOA layers (p)
    num_qaoa_layers = 1 # Start with 1, can increase for better results

    # Run the optimization
    min_energy, best_params, most_common_bitstring, selected_assets = run_portfolio_optimization_qaoa(
        tickers=my_tickers,
        start_date=start_date,
        end_date=end_date,
        num_assets_to_select=k_assets_to_select,
        num_qaoa_layers=num_qaoa_layers,
        sweep_size=5, # Reduce sweep size for faster execution during testing
        repetitions=500 # Number of measurement repetitions per parameter set
    )

[*********************100%***********************]  5 of 5 completed

Fetching data for tickers: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'] from 2020-01-01 to 2023-01-01...
Data fetched and processed successfully.
Ising Hamiltonian coefficients generated.
QAOA circuit constructed with 1 layers.
Starting QAOA parameter sweep with 25 combinations and 500 repetitions per combination...



Optimization complete.
Minimum Energy Found: -2.2600407225146886
Best Parameters: OrderedDict([('gamma_0', 0.0), ('beta_0', 1.5707963267948966)])
Most common measurement outcome (bitstring): 01001 (occurred 24 times)
Selected asset indices (based on most common bitstring): [0, 2, 3]
Corresponding tickers:
- AAPL
- GOOGL
- AMZN
